In [13]:
# Definición de variables 
import pandapower.networks as pp_net
import pandapower as pp
# cargamos el modelo eléctrico 
net = pp_net.case57()
#net.sn_mva = 1.0
pp.runpp(net)
net

This pandapower network includes the following parameter tables:
   - bus (57 elements)
   - load (42 elements)
   - gen (6 elements)
   - shunt (3 elements)
   - ext_grid (1 element)
   - line (63 elements)
   - trafo (17 elements)
   - poly_cost (7 elements)
   - bus_geodata (57 elements)
 and the following results tables:
   - res_bus (57 elements)
   - res_line (63 elements)
   - res_trafo (17 elements)
   - res_ext_grid (1 element)
   - res_load (42 elements)
   - res_shunt (3 elements)
   - res_gen (6 elements)

In [16]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,103.227389,74.685047,-101.897941,-83.756126,1.329448,-9.071079,0.615060,0.655652,0.655652,1.040000,0.000000,1.010000,-1.206976,1.319157
1,98.897941,-4.930366,-96.040525,4.940285,2.857416,0.009920,0.492205,0.490155,0.492205,1.010000,-1.206976,0.985000,-6.067217,0.990307
2,62.096390,-6.730933,-61.648511,4.526199,0.447879,-2.204734,0.318352,0.316656,0.318352,0.985000,-6.067217,0.980040,-7.448772,0.640518
3,14.207067,-4.974233,-14.066647,2.802334,0.140420,-2.171899,0.077110,0.073760,0.077110,0.980040,-7.448772,0.976248,-8.711721,0.155144
4,14.850721,-5.761077,-14.744497,2.784357,0.106225,-2.976719,0.081599,0.076870,0.081599,0.980040,-7.448772,0.980000,-8.865008,0.164176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,-7.174074,-3.001910,7.688997,3.513108,0.514923,0.511198,0.024223,0.024223,0.024223,0.805922,-17.393973,0.876054,-15.544598,0.097471
59,-2.398253,-2.351463,2.435161,2.412947,0.036908,0.061484,0.010461,0.010461,0.010461,0.805922,-17.393973,0.822586,-17.092087,0.042096
60,-3.973397,-1.274103,4.021578,1.346098,0.048181,0.071994,0.013209,0.013209,0.013209,0.792961,-18.121449,0.805922,-17.393973,0.053153
61,-3.663896,-2.229309,3.692696,2.056485,0.028800,-0.172824,0.021208,0.020670,0.021208,0.846059,-13.680227,0.855491,-13.360033,0.051204


In [18]:
net.line

,c_nf_per_km,df,from_bus,g_us_per_km,in_service,length_km,max_i_ka,max_loading_percent,name,parallel,r_ohm_per_km,std_type,to_bus,type,x_ohm_per_km
0,2587.396050,1.0,0,0.0,True,1.0,49.702328,100.0,None,1,1.097675,None,1,ol,3.703000
1,1640.689899,1.0,1,0.0,True,1.0,49.702328,100.0,None,1,3.941050,None,2,ol,11.241250
2,762.178681,1.0,2,0.0,True,1.0,49.702328,100.0,None,1,1.481200,None,3,ol,4.840350
3,517.479210,1.0,3,0.0,True,1.0,49.702328,100.0,None,1,8.265625,None,4,ol,17.457000
4,697.995214,1.0,3,0.0,True,1.0,49.702328,100.0,None,1,5.686750,None,5,ol,19.573000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.000000,1.0,55,0.0,True,1.0,24.851164,100.0,None,1,292.537000,None,40,ol,290.421000
59,0.000000,1.0,55,0.0,True,1.0,24.851164,100.0,None,1,112.412500,None,41,ol,187.266000
60,0.000000,1.0,56,0.0,True,1.0,24.851164,100.0,None,1,92.046000,None,55,ol,137.540000
61,41.786112,1.0,37,0.0,True,1.0,41.418606,100.0,None,1,21.900600,None,48,ol,33.707880


In [19]:
net.bus

,in_service,max_vm_pu,min_vm_pu,name,type,vn_kv,zone
0,True,1.06,0.94,1,b,115.0,1.0
1,True,1.06,0.94,2,b,115.0,1.0
2,True,1.06,0.94,3,b,115.0,1.0
3,True,1.06,0.94,4,b,115.0,1.0
4,True,1.06,0.94,5,b,115.0,1.0
5,True,1.06,0.94,6,b,115.0,1.0
6,True,1.06,0.94,7,b,115.0,1.0
7,True,1.06,0.94,8,b,115.0,1.0
8,True,1.06,0.94,9,b,115.0,1.0
9,True,1.06,0.94,10,b,115.0,1.0


In [5]:
from pyomo.environ import *

model = ConcreteModel()
model.i = Set(initialize=[1,2,3,4], doc='Buses')
model.j = Set(initialize=[1,2,3,4], doc='Buses')
model.c = Set(initialize=[1,2,3], doc='Lines')
model.t = Set(initialize=[1,2,3,4,5], doc='Time')

model.V_LineP = Var(model.i, model.j, model.c, model.t, within=NonNegativeReals, doc='Real power flowing from bus i towards bus j on line c at time t')


In [21]:
model.Nodes = Set()

In [ ]:
model.Arcs = Set(within=model.Nodes*model.Nodes)

In [19]:
model.Arcs.display()

Arcs : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain      : Size : Members
    None :     2 : Arcs_domain :    0 :      {}


In [7]:
branchstatus = dict(
    ((net.line.iloc[c]['from_bus'],net.line.iloc[c]['to_bus'],c), 
    net.line.iloc[c]['in_service']) 
    for c in range(net.line.shape[0])
    )

In [ ]:
from pyomo.environ import *

model = ConcreteModel()

model.x = Var(model.i, model.j, model.c,model.t, within=Binary)

model.V_LineP = Var(model.i, model.j, model.c,model.t, bounds=(0,1000),doc='Real power flowing from bus i towards bus j on line c at time t')

# Set the allowed combinations
allowed_combinations = [ (0,1,2,3), (1,2,3,4), (4,5,6,7) ]

def V_LineP_restriction(model, i, j, c, t):
    if (i,j,c,t) in allowed_combinations:
        return model.x[i,j,c,t]*model.V_LineP[i,j,c,t] <=1000
    else:
        return Constraint.Skip

model.V_LineP_restriction = Constraint(model.i, model.j, model.c,model.t, rule=V_LineP_restriction)

def only_one_active(model):
    return sum(model.x[i,j,c,t] for i,j,c,t in allowed_combinations) <= 1

model.only_one_active = Constraint(rule=only_one_active)
